# Privacy Preserving Training and Inference Machine Learning over Medical Data 

## Introduction
In this tutorial, we will demonstrate the process of privacy-preserving training using Fed-BioMed, which leverages federated learning with secure aggregation. Subsequently, we will deploy the final model obtained through federated learning in a privacy-preserving manner using the Concrete-ML library. This approach allows us to achieve privacy-preserving inference through a software-as-a-service (SaaS) model.

The selected dataset originates from a medical task assigned by Flamby, specifically the FedHeart Disease dataset. For more detailed information about the dataset, please refer to the provided link.

## Install Concrete-ML

This tutorial assumes you have Concrete-ML installed in your Fed-BioMed researcher environment. 

If needed, you may install it with:
1. `source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment researcher`
2. `pip install concrete-ml`

### Note for MacOS users with ARM chips

If you have a recent Mac machine with Apple Silicon (ARM chips), then you may experience kernel failure in Section 3.2 of this notebook. 

To overcome this issue, you need to rebuild your conda `fedbiomed-researcher` environment with a native python executable, before installing Concrete-ML. This process may take some time. 

1. `export CONDA_SUBDIR=osx-arm64`
2. `${FEDBIOMED_DIR}/scripts/configure_conda -c researcher`

##  Flamby configuration - Download FedHeart

You need to download the FLamby dataset that we will use. For licensing reasons, these are not including directly in the FLamby installation.

To download the fed_heart dataset in `${FEDBIOMED_DIR}/data` (where `${FEDBIOMED_DIR}` is the base directory of Fed-BioMed): 

1. `source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment node`<br>
2. `pip install wget`<br>
3. `python ${FEDBIOMED_DIR}/docs/tutorials/concrete-ml/download.py --output-folder ${FEDBIOMED_DIR}/data`

In [1]:
import os
import torch 
from torch.utils.data import DataLoader
import numpy as np
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_heart_disease import FedHeartDisease
from fedbiomed.common.data import FlambyDataset, DataManager
from concrete.ml.torch.compile import compile_torch_model
from torch.nn.modules.loss import _Loss
import torch.nn as nn
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [2]:
FEDBIOMED_DIR = os.getenv('FEDBIOMED_DIR')
DATASET_TEST_PATH = f"{FEDBIOMED_DIR}/data"

## 1. Fed-BioMed

Configuring the Fed-BioMed training plan involves specifying the machine learning model, defining the loss function, and identifying the necessary dependencies. This ensures a clear and well-defined setup for the training process.

In [3]:
%load_ext tensorboard

In [4]:
class FedHeartTrainingPlan(TorchTrainingPlan):
    
    class Baseline(nn.Module):
        
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(13, 16)
            self.fc2 = nn.Linear(16, 2)
            self.act = nn.LeakyReLU()
        def forward(self, x):
            x = self.act(self.fc1(x))
            x = self.fc2(x)
            return x
        
    class BaselineLoss(_Loss):
        def __init__(self):
            super().__init__()
            self.ce = torch.nn.CrossEntropyLoss()

        def forward(self, prediction: torch.Tensor, target: torch.Tensor):
            target = torch.squeeze(target, dim=1).type(torch.long)
            return self.ce(prediction, target)
    
    def init_model(self, model_args):
        return self.Baseline()

    def init_optimizer(self, optimizer_args):
        return torch.optim.AdamW(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_heart_disease import FedHeartDisease",
                "from torch.nn.modules.loss import _Loss",
                "from fedbiomed.common.data import FlambyDataset, DataManager"]

    def training_step(self, data, target):
        logits = self.model().forward(data)
        return self.BaselineLoss().forward(logits, target)

    def training_data(self, batch_size=2):
        dataset = FlambyDataset()
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset, **train_kwargs)

In [5]:
batch_size=8
num_updates=10
def get_nb_max_rounds(num_updates, batch_size, num_epochs_pooled):
    return (486 // 4 // batch_size) * num_epochs_pooled // num_updates
num_rounds = get_nb_max_rounds(num_updates=num_updates, batch_size=batch_size, num_epochs_pooled=50)
num_rounds
print(num_rounds)

75


In [6]:
training_args = {
    'optimizer_args': {
        'lr': 5e-4,
    },
    'loader_args': {
        'batch_size': batch_size,
    },
    'num_updates': num_updates,
    'dry_run': False,
    'log_interval': 2,
    'test_ratio' : 0.0,
    'test_on_global_updates': False,
    'test_on_local_updates': False,
    'random_seed':42,
}

model_args = {}

## 2. Federated Learning Training with SecAgg
⚠️ Before executing the following cell (if `TRAIN==True`), make sure to configure the nodes using the steps below:

**Nodes Configuration:**
For each node in the range `[0..3]`:

1. Open a new terminal.
2. Run the command: `./scripts/fedbiomed_run node --config config-n{i}.ini dataset add`
3. Select `6) flamby`.
4. Enter the dataset name: `flamby` (optional).
5. Set tags to: `heart` (important).
6. Description: Enter `none` (optional).
7. Select `1) fed_heart_disease`.
8. Specify a center ID between 0 and 3: `{i}`.
9. Description: Enter `none` (optional).
10. Run the command: `./scripts/fedbiomed_run node --config config-n{i}.ini start`.

### Configuring Secure Aggregation

If you haven't done so already, you need to configure SecAgg in Fed-BioMed. 
A quick reminder of the basic commands is given below, but please refer to our documentation for a full explanation.

1. `${FEDBIOMED_DIR}/scripts/fedbiomed_configure_secagg researcher`
2. make sure the researcher and nodes' config files have been created (e.g. step 2 above)
3. `${FEDBIOMED_DIR}/scripts/fedbiomed_run certificate-dev-setup`


In [8]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['heart']

exp_sec_agg = Experiment(tags=tags,
                 training_plan_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 model_args=model_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                 secagg=True,
                 tensorboard=True
                )

2024-05-15 13:49:47,459 fedbiomed WARNING - Importing Experiment class from the researcher package has been deprecated. Please use the following import statement instead:
from fedbiomed.researcher.federated_workflows import Experiment

2024-05-15 13:49:47,463 fedbiomed INFO - Starting researcher service...

2024-05-15 13:49:47,467 fedbiomed INFO - Waiting 3s for nodes to connect...

2024-05-15 13:49:49,256 fedbiomed DEBUG - Node: NODE_1b872dd2-06cd-4206-9bdf-c2ccd2b7f688 polling for the tasks

2024-05-15 13:49:49,258 fedbiomed DEBUG - Node: NODE_7834355d-26b7-4404-b270-81843f10b715 polling for the tasks

...

2024-05-15 13:49:50,503 fedbiomed INFO - Node selected for training -> NODE_7834355d-26b7-4404-b270-81843f10b715

2024-05-15 13:49:50,503 fedbiomed INFO - Node selected for training -> NODE_95dafd89-1f88-4326-91ed-8d428088a0bb

2024-05-15 13:49:50,510 fedbiomed DEBUG - Model file has been saved: /Users/fcremone/dev/fedbiomed/var/experiments/Experiment_0087/model_6595dd68-cef3-42c7-87da-ec25abf9f1cf.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


2024-05-15 13:49:50,519 fedbiomed INFO - Removing tensorboard logs from previous experiment

In [9]:
tensorboard --logdir "$tensorboard_dir"

In [10]:
exp_sec_agg.run()

2024-05-15 13:49:56,237 fedbiomed DEBUG - Secagg context for default_biprime0 is already existing on researcher researcher_id='RESEARCHER_9b981f0c-84eb-4725-b7c3-613bbabd19b7'

2024-05-15 13:49:56,251 fedbiomed DEBUG - Node: NODE_1b872dd2-06cd-4206-9bdf-c2ccd2b7f688 polling for the tasks

2024-05-15 13:49:56,252 fedbiomed DEBUG - Node: NODE_7834355d-26b7-4404-b270-81843f10b715 polling for the tasks

2024-05-15 13:49:56,253 fedbiomed DEBUG - Node: NODE_95dafd89-1f88-4326-91ed-8d428088a0bb polling for the tasks

2024-05-15 13:49:56,284 fedbiomed DEBUG - Node: NODE_1b872dd2-06cd-4206-9bdf-c2ccd2b7f688 polling for the tasks

...

2024-05-15 13:50:29,265 fedbiomed DEBUG - Aggregation is completed in 0.0 seconds.

2024-05-15 13:50:29,265 fedbiomed INFO - Validation is completed.

2024-05-15 13:50:29,266 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.

2024-05-15 13:50:29,283 fedbiomed INFO - Aggregating 3 parameters from 3 nodes.

2024-05-15 13:50:29,284 fedbiomed DEBUG - Aggregation is completed in 0.02 seconds.

# 3. Inference

In [22]:
fed_sec_agg_model = exp_sec_agg.training_plan().model()
fed_sec_agg_model.eval()

Baseline(
  (fc1): Linear(in_features=13, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
  (act): LeakyReLU(negative_slope=0.01)
)

In [13]:
test_dataset = FedHeartDisease(center=0,pooled=True, train=False, data_path=DATASET_TEST_PATH)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## 3.1. Inference with torch (over a plaintext model)

In [14]:
def test_torch(net, test_loader):
    """Test the network: measure accuracy on the test set."""

    # Freeze normalization layers
    net.eval()

    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the batches
    idx = 0
    for data, target in test_loader:
        # Accumulate the ground truth labels
        endidx = idx + target.shape[0]
        all_targets[idx:endidx] = target.numpy()

        # Run forward and get the predicted class id
        logits = torch.sigmoid(net(data))
        output = logits.argmax(1).detach().numpy()
        all_y_pred[idx:endidx] = output

        idx += target.shape[0]

    # Print out the accuracy as a percentage
    n_correct = np.sum(all_targets == all_y_pred)
    print(
        f"Test accuracy over plaintext model: "
        f"{n_correct / len(test_loader) * 100:.2f}%"
    )


In [15]:
test_torch(fed_sec_agg_model, test_dataloader)

Test accuracy over plaintext model: 78.74%


## 3.2. Inference with Concrete-ML

In [16]:
def test_with_concrete(quantized_module, test_loader, use_sim):
    """Test a neural network that is quantized and compiled with Concrete ML."""

    # Casting the inputs into int64 is recommended
    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the test batches and accumulate predictions and ground truth labels in a vector
    idx = 0
    for data, target in test_loader:
        data = data.numpy()
        target = target.numpy()

        fhe_mode = "simulate" if use_sim else "execute"

        # Quantize the inputs and cast to appropriate data type
        logits = torch.tensor(quantized_module.forward(data, fhe=fhe_mode), requires_grad=False)

        endidx = idx + target.shape[0]

        all_targets[idx:endidx] = target

        # Get the predicted class id and accumulate the predictions
        y_pred = torch.sigmoid(logits).argmax(1).numpy()
        all_y_pred[idx:endidx] = y_pred

        # Update the index
        idx += target.shape[0]
    n_correct = np.sum(all_targets == all_y_pred)
    print(
        f"Test accuracy over encrypted model: "
        f"{n_correct / len(test_loader) * 100:.2f}%"
    )


In [17]:
# concrete ml is using the traceback, 
# while fed-biomed for logging reasons fixs it to 3, to use concrete-ml we reset to the default value
import sys
sys.tracebacklimit = 1000

In [18]:
n_bits = 6
compile_set = np.random.randint(0, 10, (100, 13)).astype(float)
q_module = compile_torch_model(fed_sec_agg_model, compile_set, rounding_threshold_bits=6)

In [19]:
test_with_concrete(q_module, test_dataloader, True)

Test accuracy over encrypted model: 79.92%
